In [1]:
%%capture
!pip install requests
!pip install tiktoken

## import necessary libraries


In [2]:
from groq import Groq
from dotenv import load_dotenv
import tiktoken, requests 
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import os

# load environment variables
load_dotenv('../.env')

/opt/anaconda3/envs/llmcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Fetch data

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

print(f'Number of fetched documents: {len(documents)}')

Number of fetched documents: 948


In [4]:
# a sample of them
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
# docker takes the lead

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-hw-01-class2024"

es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:01<00:00, 937.97it/s] 


In [6]:
def elastic_search(query, size=7):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    result_scores = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        result_scores.append(hit['_score'])


    return result_docs, result_scores, response

## Answers for specific queries

In [7]:
docs, scores, response = elastic_search("How do I execute a command in a running docker container?")

In [8]:
docs[0], scores[0]

({'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 83.52589)

In [9]:
response['hits'].keys()

dict_keys(['total', 'max_score', 'hits'])

In [10]:
docs, scores, response = elastic_search("How do I execute a command in a running docker container?", 3)

In [11]:
docs[2]

{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I copy files from a different folder into docker container’s working directory?',
 'course': 'machine-learning-zoomcamp'}

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        #context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [13]:
len(build_prompt("How do I execute a command in a running docker container?", docs))

1462

In [14]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [15]:
len(encoding.encode(build_prompt("How do I execute a command in a running docker container?", docs)))

322

In [16]:
# That's a wrap